In [1]:
import tensorflow as tf;import tensorflow.keras as keras;import numpy as np;
from sklearn.datasets import fetch_california_housing;from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split;import os

In [3]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

In [5]:
datos_entrenamiento=np.c_[X_train,y_train]
datos_validacion=np.c_[X_valid,y_valid]
datos_test= np.c_[X_test,y_test]

In [18]:
def crear_archivos(data,prefijo,nombres_col=None,particiones=10):
    os.makedirs('data', exist_ok=True)
    m=len(data)
    filepaths = []
    for index,particion_data in enumerate(np.array_split(np.arange(m),particiones)):
        direccion=f'data/my_{prefijo}_{index}.csv'
        filepaths.append(direccion)
        with open(direccion,'wt',encoding='utf-8') as f:
            
            if nombres_col is not None:
                f.write(nombres_col)
                f.write("\n")
            for row_idx in particion_data:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
                
    return filepaths
        
    

In [19]:
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = crear_archivos(datos_entrenamiento, "train", header, particiones=20)



In [23]:
import pandas as pd

pd.read_csv(train_filepaths[19])

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,2.4792,24.0,3.454704,1.134146,2251.0,3.921603,34.18,-118.38,2.000
1,3.9543,35.0,5.134122,0.950690,1305.0,2.573964,33.94,-118.00,2.144
2,3.3750,8.0,5.671233,1.150685,413.0,2.828767,37.10,-119.85,0.807
3,2.9571,28.0,5.215760,0.956848,1638.0,3.073171,38.48,-121.45,1.031
4,3.6786,27.0,5.929078,1.060284,757.0,2.684397,38.64,-121.37,1.597
...,...,...,...,...,...,...,...,...,...
575,2.3375,30.0,3.371681,1.008850,840.0,3.716814,34.29,-118.45,1.542
576,4.0977,50.0,4.788009,1.036403,1171.0,2.507495,37.76,-122.48,3.221
577,1.5561,35.0,4.139932,1.220137,1920.0,3.276451,34.11,-117.29,0.680
578,2.1055,52.0,5.178495,1.070968,1361.0,2.926882,36.76,-119.79,0.613


In [25]:
file_dataset=tf.data.Dataset.list_files(train_filepaths,seed=42)

for path in file_dataset:
    print(path)


tf.Tensor(b'data\\my_train_13.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_6.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_1.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_7.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_0.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_4.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_18.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_2.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_9.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_15.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_3.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_5.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_8.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_12.csv', shape=(), dtyp

In [26]:
ciclos=5

dataset= file_dataset.interleave(lambda file_path: tf.data.TextLineDataset(file_path).skip(1),
    cycle_length=ciclos)

In [30]:
len(list(dataset.as_numpy_iterator()))

11610